Abre conexão com um banco de dados já existente:

In [1]:
import psycopg2

def conectar_banco():
  try:
    conexao = psycopg2.connect(host='localhost', database='postgres', user='postgres', password='')
  except psycopg2.DatabaseError as error:
    print (error)
  return conexao

Função para executar um comando SQL:

In [2]:
def comando_sql(SQL):
    try:
      con = conectar_banco()
      cursor = con.cursor()
      cursor.execute(SQL)
      con.commit()
      print("Comando realizado com sucesso!")
      cursor.close()
      con.close()
    except psycopg2.DatabaseError as error:
      print (error)

Função para executar um comando SQL (uma consulta). A função retorna um vetor com os resultados.

In [11]:
def consulta(SQL):
  try:
    con = conectar_banco()
    cursor = con.cursor()
    cursor.execute(SQL)
    con.commit()
    print("Comando realizado com sucesso!")
    resultado = cursor.fetchall()
    cursor.close()
    con.close()
    print('')
    print("Resultado da consulta:")
    if len(resultado) == 0:
      print("Nao ha oque mostrar")
    else:
      for i in resultado:
        print(i)
  except psycopg2.DatabaseError as error:
      print (error)

STORED PROCEDURES E VISÕES

i. Crie uma visão materializada que tem como atributos o nome das disciplinas e a quantidade
de turmas que foram ofertadas para cada uma dessas disciplinas.

In [ ]:
sql = '''
CREATE MATERIALIZED VIEW turmaview AS
SELECT d.nome , COUNT(t.cod_disc)
FROM disciplinas d, turmas t
WHERE d.cod_disc = t.cod_disc
GROUP BY d.nome
'''
comando_sql(sql)

Consulta da visão materializada:

In [ ]:
sql = 'SELECT * FROM turmaview'
consulta(sql)

ii. Defina uma função que tem como parâmetro de entrada o nome de um professor e como saída
todas as turmas e o respectivo ano em que elas foram ministradas por esse professor.

In [ ]:
sql = '''
CREATE OR REPLACE FUNCTION mostrar_turmas_ano(nome_prof varchar)
returns table(cod_turma int, ano varchar)
language plpgsql
as $$
begin
    return query SELECT t.cod_turma, t.ano 
    FROM  Professores p, Turmas t
    WHERE p.nome = nome_prof AND p.cod_prof = t.cod_prof;

end;$$
'''

comando_sql(sql)

Insira um nome de um professor entre as 'aspas' para o teste:

In [27]:
nome_prof = 'insira_aqui'

Chamando a função e executando:

In [ ]:
sql = "SELECT * FROM mostrar_turmas_ano('%s')" % nome_prof
consulta(sql)

iii. Defina uma função que tem como parâmetro de entrada o nome de um aluno e como saída o
nome das disciplinas e a sua maior nota em cada disciplina que constam no seu histórico.

In [ ]:
sql = '''
CREATE OR REPLACE FUNCTION mostrar_disc_nota(nome_alu varchar)
returns table (nome varchar, nota int)
language plpgsql
as $$
begin
    return query SELECT d.nome, MAX(h.nota) 
    FROM disciplinas d, historico h, alunos a 
    WHERE a.nome = nome_alu AND a.mat = h.mat AND d.cod_disc = h.cod_disc
    GROUP BY d.nome; 
end;$$
'''

comando_sql(sql)

Insira um nome de um aluno entre as 'aspas' para o teste:

In [29]:
nome_alu = 'insira_aqui'

Chamando a função e executando:

In [ ]:
sql = "SELECT * FROM mostrar_disc_nota('%s')" % nome_alu
consulta(sql)